In [19]:
from pipeline import *
import pandas as pd

In [20]:
year = 10

df_teams, df_teams_post, df_series_post, df_players, df_players_teams, df_coaches, df_awards_players = load_data()

df_final = global_merge(df_teams, df_teams_post, df_series_post,
                                   df_players, df_players_teams, df_coaches, df_awards_players, year)

df_final.head(10)

,tmID,year,offensive_strength,defensive_strength,confID,playoff,win_percentage_coach,awards
0,ATL,9,0.174422,0.149886,EA,0,-2.235235,2
1,ATL,10,0.577290,0.617877,EA,1,-2.235235,5
2,CHA,1,0.079772,0.064578,EA,0,-1.295938,2
3,CHA,2,0.673662,0.656730,EA,1,0.533667,3
4,CHA,3,0.682665,0.659045,EA,1,0.533667,3
5,CHA,4,0.814721,0.825337,EA,1,-0.478402,3
6,CHA,5,0.889861,0.882206,EA,0,-0.478402,2
7,CHA,6,1.265729,1.413766,EA,0,0.000000,3
8,CHA,7,0.798807,0.972582,EA,0,-0.857600,1
9,CHI,7,0.483068,0.573005,EA,0,-2.026503,0


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier


# Define a list of lambda functions for different models with hyperparameters
models = [
    lambda: RandomForestClassifier(n_estimators=100, random_state=42),
    #lambda: LogisticRegression(max_iter=1000, random_state=42),
    lambda: SVC(C=1.0, kernel='rbf', probability=True),
    lambda: GaussianNB(),
    lambda: KNeighborsClassifier(n_neighbors=5),
    lambda: DecisionTreeClassifier(max_depth=5, random_state=42),
    lambda: GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    lambda: MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
]

In [24]:
from sklearn.metrics import accuracy_score



def expanding_window_decay_cross_validation(data, model_func, features, n_training_years, decay_rate=0.9):
    accuracy_results = []
    model = model_func()

    # from final take only numeric columns
    data_num = data.select_dtypes(include=['int64', 'float64'])

    for i in range(1, n_training_years):
        # train_data = data[data['year'].between(1, i)]
        train_data = data_num[data_num['year'].between(1, i)]
        test_data = data_num[data_num['year'] == (i + 1)]

        X_train, X_test = train_data[features], test_data[features]
        y_train, y_test = train_data['playoff'], test_data['playoff']

        # print(X_train, X_test)

        # Apply weight to older data
        weight = decay_rate ** (n_training_years - i - 1)
        sample_weight = [weight] * len(X_train)

        if type(model_func()).__name__ in ["KNeighborsClassifier", "MLPClassifier"]:
            model.fit(X_train, y_train)  # This model don't support sample weights
        else:
            model.fit(X_train, y_train, sample_weight=sample_weight)

        predictions = model.predict(X_test)

        test_data['predictions'] = predictions
        data['predictions'] = 0
        data.loc[data['year'] == year, 'predictions'] = predictions

        print(data)

        _, ea_teams, we_teams = classify_playoff_entry(
            data, year)

        ea_predictions = ea_teams['tmID'].unique()
        we_predictions = we_teams['tmID'].unique()

        accuracy = calculate_playoff_accuracy(
            year, ea_predictions, we_predictions, display_results=False)
        accuracy_results.append(accuracy)

    return accuracy_results, model


In [25]:


n_training_years = 9 # 9 years of data and 1 year to predict

for model in models:
    accuracy_results, final_model = expanding_window_decay_cross_validation(
        df_final, model,df_final.columns , n_training_years)

    avg = round(sum(accuracy_results) / len(accuracy_results), 2)

    print(f"=====================================")
    print("Model name:", type(model()).__name__)
    print("Accuracy results for each iteration:",
          [round(x, 2) for x in accuracy_results])
    print("Average accuracy:", avg)

print("=====================================")

KeyError: "['tmID', 'confID'] not in index"